# Решение спитцеровской теплопроводности в плазме

## Неявный метод с неоднородностью на нижнем слое

In [ ]:
from scipy.optimize import root
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
def qei(Te, Ti):
    if Te == 0:
        return 0.0
    else:
        return (Te - Ti) / Te ** 2

In [ ]:
def f_electron_ion(z, u1, u2, n, tau, h, M, kappa=0.2, alpha=2.5):
    Func = np.zeros(M, dtype=np.double)
    # Задаём уравнение с граничным условием для электронной температуры
    k1 = 0.5 * (z[0] ** alpha + z[1] ** alpha)
    Func[0] = (z[0] - u1[n, 0]) / tau - kappa / h * (k1 * (z[1] - z[0]) / h) - qei(u1[n, 0], u2[n, 0])
    # Задаём уравнения для электронной температуры
    for m in range(1, M - 1):
        k1 = 0.5 * (z[m] ** alpha + z[m + 1] ** alpha)
        k2 = 0.5 * (z[m] ** alpha + z[m - 1] ** alpha)
        Func[m] =  (z[m] - u1[n, m]) / tau - kappa / h * (k1 * (z[m + 1] - z[m]) / h - k2 * (z[m] - z[m - 1]) / h) - qei(u1[n, m], u2[n, m])
    # Заполняем предпоследний узел
    m = M - 1
    k1 = 0.5 * z[m] ** alpha
    k2 = 0.5 * (z[m] ** alpha + z[m - 1] ** alpha)
    Func[m] = (z[m] - u1[n, m]) / tau - kappa / h * (k1 * (0 - z[m]) / h - k2 * (z[m] - z[m - 1]) / h) - qei(u1[n, m], u2[n, m])
    return Func

In [ ]:
def j_electron_ion(z, u1, u2, n, tau, h, M, kappa=0.2, alpha=2.5):
    J = np.zeros((M, M), dtype=np.double)
    # Запишем частные производные для первого уравнения электронной температуры
    J[0, 0] = 1 / tau - kappa / (2 * h ** 2) * (alpha * z[0] ** (alpha - 1) * (z[1] - z[0]) - (z[0] ** alpha + z[1] ** alpha))
    J[0, 1] = - kappa / (2 * h ** 2) * (alpha * z[1] ** (alpha - 1) * (z[1] - z[0]) + (z[0] ** alpha + z[1] ** alpha))
    # Запишем частные производные для уравнений электронной температуры
    for i in range(1, M - 1):
        J[i,i - 1] = kappa / (2 * h ** 2) * (alpha * z[i - 1] ** (alpha - 1) * (z[i] - z[i - 1]) - (z[i - 1] ** alpha + z[i] ** alpha))
        J[i, i] = 1 / tau - kappa / (2 * h ** 2) * (alpha * z[i] ** (alpha - 1) * (z[i + 1] - z[i]) - (z[i] ** alpha + z[i + 1] ** alpha) - alpha * z[i] ** (alpha - 1) * (z[i] - z[i - 1]) - (z[i - 1] ** alpha + z[i] ** alpha))
        J[i,i + 1] = - kappa / (2 * h ** 2) * (alpha * z[i + 1] ** (alpha - 1) * (z[i + 1] - z[i]) + (z[i] ** alpha + z[i + 1] ** alpha))
    # Запишем частные производные для последнего уравнения электронной температуры
    i = M - 1
    J[i,i - 1] = kappa / (2 * h ** 2) * (alpha * z[i - 1] ** (alpha - 1) * (z[i] - z[i - 1]) - (z[i - 1] ** alpha + z[i] ** alpha))
    J[i, i] = 1 / tau - kappa / (2 * h ** 2) * (alpha * z[i] ** (alpha - 1) * (0 - z[i]) - (z[i] ** alpha + 0) - alpha * z[i] ** (alpha - 1) * (z[i] - z[i - 1]) - (z[i - 1] ** alpha + z[i] ** alpha))
    return J

In [ ]:
# Начальные значения
a = 0
b = 2
t_0 = 0
T = 0.01
To = 2

In [ ]:
# Число узлов
N = 6000
M = 400

# Размер шага сетки
h = (b - a) / M
tau = (T - t_0) / N
print(f'{h=}, {tau=: .4f}')

tau / h ** 2 * 0.5

In [ ]:
t = np.linspace(t_0, T, N + 1)
x = np.linspace(a, b, M + 1)

In [ ]:
def u_init(x, To):
    u = np.zeros(x.size)
    for i in range(len(x)):
        if x[i] <= 1:
            u[i] = To
    return u

In [ ]:
u1 = np.zeros((N + 1, M + 1), dtype=np.double)
u1[0, :] = u_init(x, 2)

u2 = np.zeros((N + 1, M + 1), dtype=np.double)
u2[0, :] = u_init(x, 2)

In [ ]:
u1[1, :]

In [ ]:
from tqdm.notebook import tqdm
import time

In [ ]:
for n in tqdm(range(N)):
    res_electron = root(fun=f_electron_ion, x0=u1[n, :-1], args=(u1, u2, n, tau, h, M, 0.2, 2.5), method='krylov', jac=j_electron_ion)
    res_ion = root(fun=f_electron_ion, x0=u2[n, :-1], args=(u1, u2, n, tau, h, M, 0.3, 1.5), method='krylov', jac=j_electron_ion)
    u1[n + 1, :-1] = res_electron.x
    u2[n + 1, :-1] = res_ion.x

In [ ]:
res_electron.x

In [ ]:
np.all(u1[1, :] == u1[2, :])

In [ ]:
from matplotlib import pyplot as plt
from matplotlib.pyplot import axes
from celluloid import Camera

fig = plt.figure()
camera = Camera(fig)
ax = axes(xlim=(0, 1.5), ylim=(0, 2.5))

for i in range(0, N, 20):
    ax.plot(x, u1[i, :], color="blue")
    ax.plot(x, u2[i, :], color="red")
    camera.snap()
animation = camera.animate()
animation.save('oh_my.gif', writer = 'imagemagick')

In [ ]:
from IPython.display import Image
Image(url='oh_my.gif')

In [ ]:
import numpy as np
from matplotlib import pyplot as plt
from matplotlib.animation import FuncAnimation
plt.style.use('seaborn-pastel')

fig = plt.figure()
ax = plt.axes(xlim=(0, 4), ylim=(-2, 2))
line, = ax.plot([], [], lw=3)

def init():
    line.set_data([], [])
    return line,
def animate(i):
    x = np.linspace(0, 4, 1000)
    y = np.sin(2 * np.pi * (x - 0.01 * i))
    line.set_data(x, y)
    return line,

anim = FuncAnimation(fig, animate, init_func=init,
                               frames=200, interval=20, blit=True)

anim.save('sine_wave.gif', writer='imagemagick')

# Решение одномерного квазилинейного уравнения

In [ ]:
import numpy as np
from matplotlib import pyplot as plt
from matplotlib.animation import FuncAnimation
plt.style.use('seaborn-pastel')

fig = plt.figure()
ax = plt.axes(xlim=(0, 4), ylim=(-2, 2))
line, = ax.plot([], [], lw=3)

def init():
    line.set_data([], [])
    return line,
def animate(i):
    x = np.linspace(0, 4, 1000)
    y = np.sin(2 * np.pi * (x - 0.01 * i))
    line.set_data(x, y)
    return line,

anim = FuncAnimation(fig, animate, init_func=init,
                               frames=200, interval=20, blit=True)

anim.save('sine_wave.gif', writer='imagemagick')

## Решение при помощи потоковой прогонки

In [ ]:
def K_1_2(y_m, y_m_1, alpha):
    return (y_m ** alpha + y_m_1 ** alpha) / 2

In [ ]:
def W_1_2(y_m, y_m_1, h):
    